<a href="https://colab.research.google.com/github/azzindani/01_Data_Analytic/blob/main/India_Crop_Production_Draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PENDING

## 00 Importing Modules & Mounting Drive

In [1]:
!pip install pandas-profiling
!pip install streamlit
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.445s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [2]:
import pathlib
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from ydata_profiling import ProfileReport
from tabulate import tabulate
from time import sleep

%matplotlib inline

In [3]:
'''from google.colab import drive
drive.mount('/content/drive') #'''

"from google.colab import drive\ndrive.mount('/content/drive') #"

In [4]:
'''MAIN_PATH = str(pathlib.Path().resolve())
WORK_PATH = MAIN_PATH + '/drive/MyDrive/Workspace'
SOURCE_PATH = WORK_PATH + '/00_Data_Source'
CACHE_PATH = WORK_PATH + '/00_Cache_Data' #'''

"MAIN_PATH = str(pathlib.Path().resolve())\nWORK_PATH = MAIN_PATH + '/drive/MyDrive/Workspace'\nSOURCE_PATH = WORK_PATH + '/00_Data_Source'\nCACHE_PATH = WORK_PATH + '/00_Cache_Data' #"

## 01 Choosing & Importing Dataset

### 01.01 Importing Main Data

In [5]:
dataset_path = 'https://raw.githubusercontent.com/azzindani/00_Data_Source/main/India_Crop_Production.csv'
df = pd.read_csv(dataset_path)#, encoding = 'ISO-8859-1')#, sep = '\t')
df = df.rename(columns = {'State Name' : 'State'})
df.head(2)

,Dist Code,Year,State Code,State,Dist Name,RICE AREA (1000 ha),RICE PRODUCTION (1000 tons),RICE YIELD (Kg per ha),WHEAT AREA (1000 ha),WHEAT PRODUCTION (1000 tons),...,SUGARCANE YIELD (Kg per ha),COTTON AREA (1000 ha),COTTON PRODUCTION (1000 tons),COTTON YIELD (Kg per ha),FRUITS AREA (1000 ha),VEGETABLES AREA (1000 ha),FRUITS AND VEGETABLES AREA (1000 ha),POTATOES AREA (1000 ha),ONION AREA (1000 ha),FODDER AREA (1000 ha)
0,1,1978,14,Chhattisgarh,Durg,612.5,362.2,591.35,48.7,25.6,...,2000.0,0.0,0.0,0.0,4.75,8.67,13.41,0.04,0.43,0.34
1,1,1979,14,Chhattisgarh,Durg,616.8,330.6,535.99,44.6,17.8,...,1000.0,0.0,0.0,0.0,4.76,8.62,13.38,0.06,0.48,0.28


### 01.02 Importing Geo Data

In [6]:
gdf = gpd.read_file('https://raw.githubusercontent.com/adarshbiradar/maps-geojson/master/india.json')
gdf = gdf.rename(columns = {'st_nm' : 'State'})
gdf.head(2)

,State,geometry
0,Andaman and Nicobar Islands,"MULTIPOLYGON (((93.84831 7.24028, 93.92705 7.0..."
1,Arunachal Pradesh,"POLYGON ((95.23643 26.68105, 95.19594 27.03612..."


### 01.03 Importing Additional Data (for enrichment)

## 02 Checking Data

In [7]:
df.shape

(12418, 80)

In [8]:
df.columns

Index(['Dist Code', 'Year', 'State Code', 'State', 'Dist Name',
       'RICE AREA (1000 ha)', 'RICE PRODUCTION (1000 tons)',
       'RICE YIELD (Kg per ha)', 'WHEAT AREA (1000 ha)',
       'WHEAT PRODUCTION (1000 tons)', 'WHEAT YIELD (Kg per ha)',
       'KHARIF SORGHUM AREA (1000 ha)',
       'KHARIF SORGHUM PRODUCTION (1000 tons)',
       'KHARIF SORGHUM YIELD (Kg per ha)', 'RABI SORGHUM AREA (1000 ha)',
       'RABI SORGHUM PRODUCTION (1000 tons)', 'RABI SORGHUM YIELD (Kg per ha)',
       'SORGHUM AREA (1000 ha)', 'SORGHUM PRODUCTION (1000 tons)',
       'SORGHUM YIELD (Kg per ha)', 'PEARL MILLET AREA (1000 ha)',
       'PEARL MILLET PRODUCTION (1000 tons)', 'PEARL MILLET YIELD (Kg per ha)',
       'MAIZE AREA (1000 ha)', 'MAIZE PRODUCTION (1000 tons)',
       'MAIZE YIELD (Kg per ha)', 'FINGER MILLET AREA (1000 ha)',
       'FINGER MILLET PRODUCTION (1000 tons)',
       'FINGER MILLET YIELD (Kg per ha)', 'BARLEY AREA (1000 ha)',
       'BARLEY PRODUCTION (1000 tons)', 'BARLEY YIELD

In [9]:
df.nunique()

Dist Code                                311
Year                                      40
State Code                                20
State                                     20
Dist Name                                311
                                        ... 
VEGETABLES AREA (1000 ha)               3539
FRUITS AND VEGETABLES AREA (1000 ha)    4833
POTATOES AREA (1000 ha)                 1845
ONION AREA (1000 ha)                     965
FODDER AREA (1000 ha)                   3820
Length: 80, dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12418 entries, 0 to 12417
Data columns (total 80 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Dist Code                                    12418 non-null  int64  
 1   Year                                         12418 non-null  int64  
 2   State Code                                   12418 non-null  int64  
 3   State                                        12418 non-null  object 
 4   Dist Name                                    12418 non-null  object 
 5   RICE AREA (1000 ha)                          12418 non-null  float64
 6   RICE PRODUCTION (1000 tons)                  12418 non-null  float64
 7   RICE YIELD (Kg per ha)                       12418 non-null  float64
 8   WHEAT AREA (1000 ha)                         12418 non-null  float64
 9   WHEAT PRODUCTION (1000 tons)                 12418 non-null  float64
 10

## 03 Data Structuring

### 03.01 Selecting & Dropping Variables

### 03.02 Cleaning Text Object

#### 03.02.01 Convert Header to Proper Text

In [11]:
for x in df.columns:
  y = x.title()
  df = df.rename(columns = {x : y})

#### 03.02.02 Strip Abnormal Spaces

In [12]:
for x in df.columns:
  if df[x].dtypes == 'object':
    try:
      df[x] = df[x].str.strip()
    except:
      pass

#### 03.02.03 Convert Object Content to Proper Text

In [13]:
for x in df.columns:
  if df[x].dtypes == 'object':
    for a in df[x].unique():
      b = a.title()
      df[x] = df[x].replace(a, b)
  else:
    pass

### 03.03 Coverting Data

#### 03.03.01 Convert to date

In [14]:
#df['Date'] = pd.to_datetime(df['Date'])

#### 03.03.02 Convert to integer

#### 03.03.03 Convert to Object (if necessary)

#### 03.03.04 Replace 0 to Nan (if necessary)

In [15]:
#df = df.replace(0, np.nan)

#### 03.03.05 Replace Nan with 0 (if necessary)

In [16]:
df = df.fillna(0)

## 04 Data Cleaning

### 04.01 Replacing Variable Component

### 04.02 Add New Variable

### 04.03 Inaccuracies

### 04.04 Handling Outliers
##### Data Distribution Check (Before)

#### 04.04.01 Using IQR (Inter Quantile Range)

In [17]:
def iqr_thres(dataframe, column, th1 = 0.25, th3 = 0.75):
    quartile1 = dataframe[column].quantile(th1)
    quartile3 = dataframe[column].quantile(th3)
    iqr = quartile3 - quartile1
    upper_limit = quartile3 + 1.5 * iqr
    lower_limit = quartile1 - 1.5 * iqr

    return lower_limit, upper_limit

In [18]:
def check_outliers_iqr(dataframe, column):
    lower_limit, upper_limit = iqr_thres(dataframe, column)
    if dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)].any(axis = None):
        return True
    else:
        return False

In [19]:
def replace_iqr(dataframe, columns, th1 = 0.25, th3 = 0.75, replace = False):
  data = []

  for column in columns:
    if df[column].dtypes == 'int64' or df[column].dtypes == 'float64':
      if column != 'Outcome':
        outliers_ = check_outliers_iqr(df, column)
        count = None
        lower_limit, upper_limit = iqr_thres(dataframe, column, th1, th3)

        if outliers_:
          count = dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)][column].count()
          if replace:
            if lower_limit < 0:
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
            else:
              dataframe.loc[(dataframe[column] < lower_limit), column] = lower_limit
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
        outliers_status = check_outliers_iqr(df, column)
        data.append([outliers_, outliers_status, count, column, lower_limit, upper_limit ])

  table = tabulate(data, headers = ['Outliers (Previously)', 'Outliers', 'Count', 'Column', 'Lower Limit', 'Upper Limit'], tablefmt = 'rst', numalign = 'right')
  print('Removing Outliers using IQR')
  print(table)

In [20]:
'''replace_iqr(
    dataframe = df,
    columns = [],
    replace = True
) #'''

'replace_iqr(\n    dataframe = df,\n    columns = [],\n    replace = True\n) #'

#### 04.04.02 Using Standard Deviation

In [21]:
def std_thres(dataframe, column):
    upper_limit = dataframe[column].mean() + 3 * dataframe[column].std()
    lower_limit = dataframe[column].mean() - 3 * dataframe[column].std()

    return lower_limit, upper_limit

In [22]:
def check_outliers_std(dataframe, column):
    lower_limit, upper_limit = iqr_thres(dataframe, column)
    if dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)].any(axis = None):
        return True
    else:
        return False

In [23]:
def replace_std(dataframe, columns, replace = False):
  data = []

  for column in columns:
    if df[column].dtypes == 'int64' or df[column].dtypes == 'float64':
      if column != 'Outcome':
        outliers_ = check_outliers_std(dataframe, column)
        count = None
        lower_limit, upper_limit = std_thres(dataframe, column)

        if outliers_:
          count = dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)][column].count()
          if replace:
            if lower_limit < 0:
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
            else:
              dataframe.loc[(dataframe[column] < lower_limit), column] = lower_limit
              dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
        outliers_status = check_outliers_std(dataframe, column)
        data.append([outliers_, outliers_status, count, column, lower_limit, upper_limit])

  table = tabulate(data, headers = ['Outlier (Previously)', 'Outliers', 'Count', 'Column', 'Lower Limit', 'Upper Limit'], tablefmt = 'rst', numalign = 'right')
  print('Removing Outliers using 3 Standard Deviation')
  print(table)

In [24]:
'''replace_iqr(
    dataframe = df,
    columns = [],
    replace = False
) #'''

'replace_iqr(\n    dataframe = df,\n    columns = [],\n    replace = False\n) #'

### 04.05 Handling Missing / Zeros / Null
##### Filling missing value (numerical only) is better using median than mean or mode

#### 04.05.01 Detecting Zero Value

In [25]:
zero_columns = []

for x in df.columns:
  if df[x].dtypes == 'int64' or df[x].dtypes == 'float64':
    if (df[x] == 0).sum() != 0:
      print(x, ':', str((df[x] == 0).sum()))
      zero_columns.append(x)

Rice Area (1000 Ha) : 842
Rice Production (1000 Tons) : 859
Rice Yield (Kg Per Ha) : 866
Wheat Area (1000 Ha) : 1761
Wheat Production (1000 Tons) : 1800
Wheat Yield (Kg Per Ha) : 1793
Kharif Sorghum Area (1000 Ha) : 3750
Kharif Sorghum Production (1000 Tons) : 3985
Kharif Sorghum Yield (Kg Per Ha) : 3968
Rabi Sorghum Area (1000 Ha) : 9277
Rabi Sorghum Production (1000 Tons) : 9333
Rabi Sorghum Yield (Kg Per Ha) : 9352
Sorghum Area (1000 Ha) : 3464
Sorghum Production (1000 Tons) : 3665
Sorghum Yield (Kg Per Ha) : 3650
Pearl Millet Area (1000 Ha) : 4651
Pearl Millet Production (1000 Tons) : 4830
Pearl Millet Yield (Kg Per Ha) : 4832
Maize Area (1000 Ha) : 1310
Maize Production (1000 Tons) : 1350
Maize Yield (Kg Per Ha) : 1379
Finger Millet Area (1000 Ha) : 8018
Finger Millet Production (1000 Tons) : 8161
Finger Millet Yield (Kg Per Ha) : 8161
Barley Area (1000 Ha) : 6310
Barley Production (1000 Tons) : 6320
Barley Yield (Kg Per Ha) : 6382
Chickpea Area (1000 Ha) : 1537
Chickpea Productio

#### 04.05.02 Detecting Nan / Non Available Value

In [26]:
nan_columns = []

for x in df.columns:
  if df[x].dtypes == 'object':
    if (df[x] == np.nan).sum() or (df[x] == '-').sum() != 0:
      print(x, ':', str((df[x] == np.nan).sum()))
      nan_columns.append(x)

#### 04.05.03 Replacing Zero with Mean (for numerical value if median value == 0), if necessary

In [27]:
'''cols = [
] #'''

'cols = [\n] #'

In [28]:
'''df = df.replace(0, np.nan)

for x in cols:
  df[x] = df[x].fillna(df[x].mean()) #'''

'df = df.replace(0, np.nan)\n\nfor x in cols:\n  df[x] = df[x].fillna(df[x].mean()) #'

#### 04.05.04 Replacing Zero with Median (for numerical value if median value != 0), if necessary

In [29]:
'''cols = [
] #'''

'cols = [\n] #'

In [30]:
'''df = df.replace(0, np.nan)

for x in cols:
  df[x] = df[x].fillna(df[x].median()) #'''

'df = df.replace(0, np.nan)\n\nfor x in cols:\n  df[x] = df[x].fillna(df[x].median()) #'

#### 04.05.05 Replacing Zero with Mode (for categorical / object value), if necessary

In [31]:
'''cols = [
] #'''

'cols = [\n] #'

In [32]:
'''df = df.replace(0, np.nan)

for x in cols:
  df[x] = df[x].fillna(df[x].mode()) #'''

'df = df.replace(0, np.nan)\n\nfor x in cols:\n  df[x] = df[x].fillna(df[x].mode()) #'

### 04.06 Handling Incomplete Data

### 04.07 Handling Data Biases

### 04.08 Handling Duplicates

In [33]:
#df = df.drop_duplicates()

##### Data Distribution Check (After)

## 05 Enriching Data
#### take other dataset, inside or outside from related dataset / business

In [34]:
a = df['State'].unique()
b = gdf['State'].unique()

for i in a:
  if i not in b:
    print(i)

print(50 * '=')

for i in b:
  if i not in a:
    print(i)

Orissa
Andaman and Nicobar Islands
Arunachal Pradesh
Chandigarh
Dadra and Nagar Haveli
Daman and Diu
Goa
Lakshadweep
Manipur
Meghalaya
Mizoram
Nagaland
Delhi
Puducherry
Sikkim
Tripura
Odisha
Jammu and Kashmir
Ladakh


In [35]:
df['State'] = df['State'].replace({
    'Orissa' : 'Odisha',
})

## 06 Data Validation
#### Verifying consistency, quality, and security of data

## 07 Exploration Data Analysis (Univariate)

### 07.01 Generate Data Profile

In [36]:
'''data_profile = ProfileReport(
    df,
    title = 'India Crop Production',
    dataset = {
        #'description' : 'This profiling report was generated for Analytics Vidhya Blog',
        #'copyright_holder' : 'Analytics Vidhya',
        #'copyright_year' : '2021',
        #'url' :  'https://www.analyticsvidhya.com/blog/',
    },
    correlations = {
        'pearson' : {'calculate' : True},
        'spearman' : {'calculate' : True},
        'kendall' : {'calculate' : True},
        'phi_k' : {'calculate' : True},
    },
)

data_profile #'''

"data_profile = ProfileReport(\n    df,\n    title = 'India Crop Production',\n    dataset = {\n        #'description' : 'This profiling report was generated for Analytics Vidhya Blog',\n        #'copyright_holder' : 'Analytics Vidhya',\n        #'copyright_year' : '2021',\n        #'url' :  'https://www.analyticsvidhya.com/blog/',\n    },\n    correlations = {\n        'pearson' : {'calculate' : True},\n        'spearman' : {'calculate' : True},\n        'kendall' : {'calculate' : True},\n        'phi_k' : {'calculate' : True},\n    },\n)\n\ndata_profile #"

## 08 Store Data to Google Drive

In [37]:
'''cache_dir = CACHE_PATH + '/_cache.csv'
print(cache_dir)
df.to_csv(cache_dir, index = False)

sleep(60) #'''

"cache_dir = CACHE_PATH + '/_cache.csv'\nprint(cache_dir)\ndf.to_csv(cache_dir, index = False)\n\nsleep(60) #"

## 09 Table, Chart Drafting & Testing

In [38]:
fig = px.colors.sequential.swatches_continuous()
#fig.show()

In [39]:
colors_1 = px.colors.sequential.Reds
colors_2 = px.colors.sequential.RdBu
explode = tuple([0.015] * 50)
latitude = 24
longitude = 84
chart_theme = 'plotly_dark'
streamlit_theme = 'streamlit'
margin = {'r' : 20, 't' : 40, 'l' : 20, 'b' : 10}
cmap = 'magma_r'
title_x = 0
title_font_size = 18

In [40]:
df.head(2)

,Dist Code,Year,State Code,State,Dist Name,Rice Area (1000 Ha),Rice Production (1000 Tons),Rice Yield (Kg Per Ha),Wheat Area (1000 Ha),Wheat Production (1000 Tons),...,Sugarcane Yield (Kg Per Ha),Cotton Area (1000 Ha),Cotton Production (1000 Tons),Cotton Yield (Kg Per Ha),Fruits Area (1000 Ha),Vegetables Area (1000 Ha),Fruits And Vegetables Area (1000 Ha),Potatoes Area (1000 Ha),Onion Area (1000 Ha),Fodder Area (1000 Ha)
0,1,1978,14,Chhattisgarh,Durg,612.5,362.2,591.35,48.7,25.6,...,2000.0,0.0,0.0,0.0,4.75,8.67,13.41,0.04,0.43,0.34
1,1,1979,14,Chhattisgarh,Durg,616.8,330.6,535.99,44.6,17.8,...,1000.0,0.0,0.0,0.0,4.76,8.62,13.38,0.06,0.48,0.28


In [41]:
var_num = 'Rice Area (1000 Ha)'
df = df[['State', 'Year', var_num]]
df.head(2)

,State,Year,Rice Area (1000 Ha)
0,Chhattisgarh,1978,612.5
1,Chhattisgarh,1979,616.8


In [42]:
linechart = pd.DataFrame(df.groupby(['Year'])[[var_num]].sum()).reset_index()
linechart['Year'] = linechart['Year'].astype(str)

# create time series chart

title = 'Yearly ' + var_num
#st.subheader(title)
fig_1 = go.Figure()
fig_1.add_trace(go.Scatter(
  x = linechart['Year'],
  y = linechart[var_num],
  mode = 'lines+markers',
  marker = {'color': colors_2[0]},
))
fig.update_traces(
  hovertemplate = '{:,.0}'
)
fig_1.update_layout(
  hovermode = 'x',
  height = 400,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
  yaxis = dict(tickformat = ',.0f'),
)
fig_1.show()

In [43]:
linechart = pd.DataFrame(df.groupby(['Year', 'State'])[[var_num]].sum()).reset_index()
linechart['Year'] = linechart['Year'].astype(str)

fig_1 = px.line(
  x = linechart['Year'],
  y = linechart[var_num],
  color = linechart['State'],
  markers = True,
)
fig_1.update_traces(
  #hovertemplate = '{:,.0f}'
)
fig_1.update_layout(
  hovermode = 'x',
  height = 400,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
  yaxis = dict(tickformat = ',.0f'),
)
fig_1.show()

In [44]:
chart_df_3 = df.groupby(by = ['State'], as_index = False,)[[var_num]].sum()
chart_df_3 = chart_df_3.sort_values(by = var_num, ascending = False)[:10]

title = '10 Highest ' + var_num + ' State'
#st.subheader(title)
fig = px.bar(
  chart_df_3,
  x = var_num,
  y = 'State',
  template = chart_theme,
  color_discrete_sequence = colors_2,
  title = title,
  text_auto = ',.0f',
)
fig.update_layout(
    height = 400,
    margin = margin,
    titlefont = dict(size = title_font_size),
    title_x = title_x,
)
fig.update_yaxes(categoryorder = 'total ascending')
fig.show()

In [45]:
var_label = 'State'
var_number = var_num

map_df = df.groupby(['State', 'Year'])[var_num].sum()
map_df = map_df.reset_index()
map_df = map_df.fillna(0)

sel_year = map_df['Year'].unique()[-1:]

#map_df['Year'] = map_df['Year'].astype('str')
#map_df['Year'] = map_df['Year'] + '-01-01'
#map_df['Year'] = pd.to_datetime(map_df['Year'])
#map_df['Year'] = map_df['Year'].dt.to_period('Y')
#map_df['Year'] = map_df['Year'].astype('str')
map_df = gdf.merge(map_df, on = 'State')
map_df = map_df[(map_df['Year'] >= sel_year.min()) & (map_df['Year'] <= sel_year.max())]

map_df = map_df.set_index(var_label)
map_df.head(2)

,geometry,Year,Rice Area (1000 Ha)
State,,,
Assam,"POLYGON ((95.19594 27.03612, 95.08795 26.94578...",2017,2417.18
Bihar,"POLYGON ((88.11357 26.54028, 88.28006 26.37640...",2017,3306.98


In [46]:
fig = px.choropleth_mapbox(
    data_frame = map_df,
    geojson = map_df.geometry,
    locations = map_df.index,
    color = var_number,
    color_continuous_scale = colors_1,
    #range_color = (0, 10),
    opacity = 0.5,
    center = {'lat' : latitude, 'lon' : longitude},
    mapbox_style = 'carto-positron',
    zoom = 3.6,
    animation_frame = 'Year',
    #width = 800,
    height = 600,
)

fig.update_geos(fitbounds = 'locations', visible = False)
fig.update_layout(margin = {'r' : 0, 't' : 0, 'l' : 0, 'b' : 0},)

fig.show()

## 10 Create Dashboard
### * template : ['ggplot2', 'seaborn', 'simple_white', 'plotly', 'plotly_white', 'plotly_dark', 'presentation', 'xgridoff', 'ygridoff', 'gridon', 'none']
### * change layout https://plotly.com/python-api-reference/generated/plotly.graph_objects.Layout.html
### * more chart customization : https://plotly.com/python-api-reference/index.html

In [47]:
%%writefile app.py

import streamlit as st
import os
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import warnings
warnings.filterwarnings('ignore')

# 00 CREATING TAB TITLE

st.set_page_config(
  page_title = 'Crop Production',
  page_icon = ':leaves:',
  layout = 'wide',
)


# 01 CREATING DASHBOARD TITLE

st.title(':leaves: India Crop Production Dashboard')
st.markdown('<style>div,block-container{padding-top:0rem;}<style>', unsafe_allow_html = True)


# 02 IMPORTING DATASET

dataset_path = 'https://raw.githubusercontent.com/azzindani/00_Data_Source/main/India_Crop_Production.csv'
df = pd.read_csv(dataset_path)#, encoding = 'ISO-8859-1')#, sep = '\t')
df = df.rename(columns = {'State Name' : 'State'})
df = df.fillna(0)

gdf = gpd.read_file('https://raw.githubusercontent.com/adarshbiradar/maps-geojson/master/india.json')
gdf = gdf.rename(columns = {'st_nm' : 'State'})

for x in df.columns:
  y = x.title()
  df = df.rename(columns = {x : y})

for x in df.columns:
  if df[x].dtypes == 'object':
    try:
      df[x] = df[x].str.strip()
    except:
      pass

for x in df.columns:
  if df[x].dtypes == 'object':
    for a in df[x].unique():
      b = a.title()
      df[x] = df[x].replace(a, b)
  else:
    pass

df['State'] = df['State'].replace({
    'Orissa' : 'Odisha',
})


# 03 SETUP TEMPLATE & THEME

colors_1 = px.colors.sequential.Greens
colors_2 = px.colors.sequential.Greens_r
explode = tuple([0.015] * 50)
latitude = 20
longitude = 84
chart_theme = 'plotly_dark'
streamlit_theme = 'streamlit'
margin = {'r' : 20, 't' : 40, 'l' : 20, 'b' : 10}
cmap = 'Greens'
title_x = 0
title_font_size = 18


col_1, col_2 = st.columns((1, 1))

# 05 CREATING FILTER

var_nums = df.columns[5:]

with col_1:
  var_num = st.selectbox('Select Variable', var_nums)

default_year = df['Year'].max()

with col_2:
  year = st.multiselect('Select Year', df['Year'].unique(), default = default_year)

if year:
  sel_df = df[df['Year'].isin(year)]
else:
  sel_df = df

df = df[['State', 'Year', var_num]]

# 07 CREATING DASHBOARD

# create highlighted indicator

col_11, col_12 = st.columns((1, 2))

chart_df_1 = df.groupby(by = ['State'], as_index = False,)[[var_num]].sum()
chart_df_1 = chart_df_1.sort_values(by = var_num, ascending = False)[:10]

with col_11:
  title = '10 Highest ' + var_num + ' State'
  #st.subheader(title)
  fig = px.bar(
    chart_df_1,
    x = var_num,
    y = 'State',
    template = chart_theme,
    color_discrete_sequence = colors_2,
    title = title,
    text_auto = ',.0f',
  )
  fig.update_layout(
      height = 600,
      margin = margin,
      titlefont = dict(size = title_font_size),
      title_x = title_x,
  )
  fig.update_yaxes(categoryorder = 'total ascending')
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)

  with st.expander('View Data'):
    st.write(chart_df_1.style.background_gradient(cmap = cmap))
    csv = chart_df_1.to_csv(index = False).encode('utf-8')
    st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

var_label = 'State'
var_number = var_num

map_df = df.groupby(['State', 'Year'])[var_num].sum()
map_df = map_df.reset_index()
map_df = map_df.fillna(0)
map_df = gdf.merge(map_df, on = 'State')
map_df = map_df.set_index(var_label)

with col_12:
  title = 'Latest ' + var_num + ' Geospatial Data'
  fig = px.choropleth_mapbox(
      data_frame = map_df,
      geojson = map_df.geometry,
      locations = map_df.index,
      color = var_number,
      color_continuous_scale = colors_1,
      #range_color = (0, 10),
      opacity = 0.5,
      center = {'lat' : latitude, 'lon' : longitude},
      mapbox_style = 'carto-positron',
      zoom = 3.6,
      #animation_frame = 'Year',
  )
  fig.update_geos(
      #fitbounds = 'locations',
      visible = False,
  )
  fig.update_layout(
      height = 600,
      margin = margin,
      title = title,
      titlefont = dict(size = title_font_size),
      title_x = title_x,
  )
  st.plotly_chart(fig, use_container_width = True, theme = streamlit_theme)


default_state = 'Assam'

state = st.multiselect('Select State', df['State'].unique(), default = default_state)

if state:
  filtered_df = df[df['State'].isin(state)]
else:
  filtered_df = df

# create time series chart

title = 'Yearly ' + var_num

linechart = pd.DataFrame(filtered_df.groupby(['Year', 'State'])[[var_num]].sum()).reset_index()
linechart['Year'] = linechart['Year'].astype(str)

fig_1 = px.line(
  x = linechart['Year'],
  y = linechart[var_num],
  color = linechart['State'],
  markers = True,
)
fig_1.update_traces(
  #hovertemplate = '{:,.0f}'
)
fig_1.update_layout(
  hovermode = 'x',
  height = 400,
  margin = margin,
  title = title,
  titlefont = dict(size = title_font_size),
  title_x = title_x,
  xaxis = dict(title = 'Year', titlefont = dict(size = 14)),
  yaxis = dict(title = var_num, titlefont = dict(size = 14), tickformat = ',.0f'),
)

st.plotly_chart(fig_1, use_container_width = True, theme = streamlit_theme)

with st.expander('View Data'):
  st.write(linechart.style.background_gradient(cmap = cmap))
  csv = linechart.to_csv(index = False).encode('utf-8')
  st.download_button('Download Data', data = csv, file_name = title + '.csv', mime = 'text/csv', help = 'Click here to download as CSV file')

Overwriting app.py


## 11 Run Dashboard

In [49]:
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com
!npx localtunnel --port 8501

35.245.110.167
npx: installed 22 in 2.5s
your url is: https://poor-spies-serve.loca.lt
^C
